<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h1>Should This Loan be Approved or Denied ?</h1>

My attempt at creating an XGBoost data model to predict whether a loan can be approved or denied.  Some notes are given to make sense of it :).
    
Any comments are welcome.
</div>

**Dataset Source**

https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

In [103]:
# filepath = "../input/should-this-loan-be-approved-or-denied/"  # Kaggle
# savepath = "./"   # Kaggle

filepath = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
savepath = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"

In [104]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
%matplotlib inline

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h3>1. Load File</h3></div>

In [93]:
sba = pd.read_csv(filepath + 'SBAnational.csv',low_memory=False)
sba.head(2)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Load file with some conversion</b><br>
    After reviewing the dataset at a glance, decided to reload with some conversions
    </div>

In [94]:
def fixvals(val):
    retval = val.replace('$','')
    retval = retval.replace(',','')
    return retval

sba = pd.read_csv(filepath + 'SBAnational.csv',\
                 converters = {'DisbursementGross':fixvals,'SBA_Appv':fixvals,\
                              'GrAppv':fixvals, 'ChgOffPrinGr':fixvals}, \
                              parse_dates=['DisbursementDate'], low_memory=False)

In [95]:
print("Shape of SBA : ", sba.shape)
sba[['DisbursementGross','SBA_Appv','GrAppv','ChgOffPrinGr','DisbursementDate']].head(2)

In [96]:
sba.head(2)

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>2. Data Preparation</h2>
    </div>

In [97]:
print(sba.columns)
print()
print(sba.info())

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>2.1 Convert dtype</h2>
    </div>

In [98]:
sba = sba.astype({'DisbursementGross':np.float64,'SBA_Appv':np.float64,\
                              'GrAppv':np.float64, 'ChgOffPrinGr':np.float64, 'NAICS':np.str_})

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>2.2 Drop rows or columns if needed</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for na's in all columns</b></div>

In [99]:
def check_cols_with_nulls(df):
    cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
    if len(cols_with_missing) == 0:
        print("No Missing Values")
    else:
        print(cols_with_missing)
    
    sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

check_cols_with_nulls(sba)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Column Drop 1 - drop columns deemed not needed</b>
    </div>

In [100]:
# Save 1
sba.to_csv(savepath + "sba_save1.csv",index=False)

In [101]:
cols_to_drop = ['LoanNr_ChkDgt', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY', \
                'ChgOffDate', 'BalanceGross']

sba.drop(columns=cols_to_drop, inplace=True)

In [102]:
sba.head(2)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>DisbursementDate</b>
    </div>

In [103]:
x = sba[sba['DisbursementDate'].isna()]
print(x.shape)

In [104]:
# Not many nulls for DisbursementDate vis-a-vis to the dataset length, so drop rows with null
sba.dropna(subset=['DisbursementDate'], how='all', inplace=True)
x = sba[sba['DisbursementDate'].isna()]
print(x.shape)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>RevLineCr</b></div>

In [105]:
len(sba[(sba['RevLineCr'] != 'Y') & (sba['RevLineCr'] != 'N')])

In [106]:
# too many unknowns, drop 'RevlineCr'
sba.drop(columns=['RevLineCr'],inplace=True)
'RevLineCR' in sba.columns

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>LowDoc</b></div>

In [107]:
sba['LowDoc'].isna().sum()

In [108]:
len(sba[(sba['LowDoc'] != 'Y') & (sba['LowDoc'] != 'N')])

In [109]:
sns.countplot(x='LowDoc',data=sba)

* **LowDoc seems to have a bearing**

In [110]:
# 5965 records that are not 'Y' or 'N', we can drop these rows
sba = sba[(sba['LowDoc'] == 'Y') | (sba['LowDoc'] == 'N')]
len(sba[(sba['LowDoc'] != 'Y') & (sba['LowDoc'] != 'N')])

In [111]:
len(sba[(sba['LowDoc'] == 'Y') | (sba['LowDoc'] == 'N')])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>NewExist</b>

In [112]:
len(sba[(sba['NewExist'] != 1) & (sba['NewExist'] != 2)])

In [113]:
sns.countplot(x='NewExist',data=sba)

In [114]:
# 1154 records that are not 1 or 2, we can drop these rows as NewExist seems to have a bearing
sba = sba[(sba['NewExist'] == 1) | (sba['NewExist'] == 2)]
len(sba[(sba['NewExist'] != 1) & (sba['NewExist'] != 2)])

In [115]:
len(sba[(sba['NewExist'] == 1) | (sba['NewExist'] == 2)])

In [116]:
# ideally, convert dtype to int32
sba = sba.astype({'NewExist':np.int32})

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>CreateJob</b></div>

In [117]:
len(sba[sba['CreateJob'].isna() == True])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>RetainedJob</b></div>

In [118]:
len(sba[sba['RetainedJob'].isna() == True])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>FranchiseCode</b></div>

In [119]:
len(sba[sba['FranchiseCode'].isna()])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>UrbanRural</b></div>

In [120]:
print(len(sba[sba['UrbanRural'].isna()]))
sba['UrbanRural'].unique()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>NoEmp</b></div>

In [121]:
len(sba[sba['NoEmp'].isna()])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Term</b></div>

In [122]:
print(len(sba[sba['Term'].isna()]))
print(len(sba[sba['Term']==0]))
print(len(sba[sba['Term']<0]))

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>City</b></div>

In [123]:
len(sba[sba['City'].isna()])

In [124]:
# drop na in City column
sba = sba[sba['City'].notna()]
len(sba[sba['City'].isna()])

In [125]:
sba.head(2)

In [126]:
# Trim leading and trailing spaces
sba['City'] = sba['City'].str.strip()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>State</b></div>

In [127]:
len(sba[sba['State'].isna()])

In [128]:
# drop na in State column
sba = sba[sba['State'].notna()]
len(sba[sba['State'].isna()])

<div style="font-family: Trebuchet MS;background-color:Chocolate;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>MIS_Status<b> - this is the **target** variable</div>

In [129]:
len(sba.loc[sba['MIS_Status'].isna()])

In [130]:
# drop na in 'MIS_Status' column
sba = sba[sba['MIS_Status'].notna()]
len(sba[sba['MIS_Status'].isna()])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>NAICS</b></div>

In [131]:
len(sba[sba['NAICS'].isna()])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Name</b> - keeping this feature temporarily, in case it is needed
    </div>

In [132]:
len(sba[sba['Name'].isna()])

In [133]:
# drop rows with na in column 'Name'
sba.dropna(subset=['Name'], how='all', inplace=True)
len(sba[sba['Name'].isna()])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for na's in all columns</b></div>

In [134]:
check_cols_with_nulls(sba)  # this function was defined earlier above

In [135]:
# Save 2
sba.to_csv(savepath + "sba_save2.csv",index=False)

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>2.3 Create New Features</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Industry</b> - The industry sector is the 1st 2 digits of NAICS
    </div>

In [136]:
sba['Industry'] = sba['NAICS'].str[0:2]
sba = sba.astype({'Industry':np.int32})

In [137]:
sba['Industry'].head(2)

In [138]:
sba['Industry'].unique()
# There is an invalid industry shown which is '0', caused by blank NAICS

In [139]:
len(sba[sba['Industry'] == 0])
# This is a bummer, as industry sector has a big effect on a business, speaking as a business 
# domain expert.  Do we drop those with NAICS = 0 ?

In [140]:
# At this stage, we leave it as is and treat it as unknown industry.  We can now drop NAICS

sba.drop(columns=['NAICS'], inplace=True)
sba.head(2)

In [141]:
# Check if we can impute from the name.  For example, a bar (or similar) business
sba[(sba['Name'].str.contains('bar',case=False)) & (sba['Industry'] == 0)]\
    [['Name','Industry']].head(10)

**It's not feasible to impute missing Industry codes efficiently, so we abandon the idea.**

In [142]:
# We can also drop 'Name' now
sba.drop(columns='Name',inplace=True)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Recession</b><br>
We want to account for variation due to the Great Recession (December 2007 to June 2009). Should we separate the datasets into different time periods ? Before, During, and After ?  Let's check how large the sets are later.  In the meantime, we create a new feature, Recession, with 1 for 'Y' and 0 for 'N' depending on the DisbursementDate. 
<br><br>
</div>

In [ ]:
# Convert "DisbursementDate" to datetime

# sba['DisbursementDate'] = pd.to_datetime(sba['DisbursementDate'], format='%d-%b-%y')

# sba.head(2)

In [143]:
# Create new column based on condition
sba['Recession'] = np.where((sba['DisbursementDate'] >= '2007-09-01')\
                     & (sba['DisbursementDate'] <= '2009-06-30'), 1, 0)

In [144]:
print(f'Total - {len(sba)}')
y = len(sba[sba['Recession'] == 1])
n = len(sba[sba['Recession'] == 0])
print(f'Yes - {y}')
print(f'No - {n}')
print(f'Yes and No - {y+n}')


<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Real Estate</b><br>
Loans backed by real estate will have terms 20 years or greater (≥240 months) and are the only loans granted for such a long term, whereas loans not backed by real estate will have terms less than 20 years ( < 240 months).<br><br>
1 - Backed By Real Estate<br>
0 - Not Backed By Real Estate<br><br>

In [145]:
# Create new column based on condition
sba['RealEstate'] = np.where(sba['Term'] >= 240, 1, 0)

In [146]:
print(f'Total - {len(sba)}')
y = len(sba[sba['RealEstate'] == 1])
n = len(sba[sba['RealEstate'] == 0])
print(f'Yes - {y}')
print(f'No - {n}')
print(f'Yes and No - {y+n}')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>SBA_Portion</b><br>
The portion which is the percentage of the loan that is guaranteed by SBA. This is derived by calculating the ratio of the amount of the loan SBA guarantees and the gross amount approved by the bank (SBA_Appv/GrAppv) * 100.<br><br></div>

In [147]:
sba['SBA_Portion']=(sba['SBA_Appv']/sba['GrAppv']) * 100
sba.head(2)

**CityState**

In [148]:
sba["CityState"] = sba["City"] + "_" + sba["State"]
sba[["CityState", "City", "State"]].head()

In [149]:
sba.head(2)

In [150]:
# Save 3
sba.to_csv(savepath + 'sba_save3.csv',index=False)

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>2.4 Encode Categorical Features</h2>
    </div>

In [151]:
sba.select_dtypes(["object"]).nunique()

<div style="font-family: Trebuchet MS;background-color:Chocolate;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>MIS_Status</b><br>
    This will be the <b>target</b> variable</div>

In [152]:
sns.set_style('whitegrid')
# Target variable is MIS Status, a categorical variable

print(sba['MIS_Status'].value_counts())
sns.countplot(x='MIS_Status',data=sba)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    This shows a skewed distribution, where this bias in the target can influence many machine learning algorithms, leading some to ignore the minority class entirely, in this case, CHGOFF.  Before oversampling the data, will try as is.<br><br></div>

In [153]:
# Update column based on condition
sba['MIS_Status'] = np.where((sba['MIS_Status'] == 'P I F'), 1, 0)

In [154]:
print(sba['MIS_Status'].dtype)
sba.head(2)[['City','MIS_Status']]

In [155]:
# ideally, convert dtype to int32
sba = sba.astype({'MIS_Status':np.int32})
sba['MIS_Status'].dtype

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>LowDoc</b><br>
'Y' = 1<br>
'N' = 0

In [156]:
# Update column based on condition
sba['LowDoc'] = np.where((sba['LowDoc'] == 'Y'), 1, 0)

sba.head(2)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Others</b></div>

In [157]:
import hashlib

cols_to_drop = []
hash_constant = 900000   # fixed value so we can programmatically reproduce the hash when needed
len_data=len(sba)
for col in sba.columns:
  if sba[col].dtype == 'object':
    print(f'Column {col} has {sba[col].nunique()} values among {len_data}')

    if sba[col].nunique() < 25:
      print(f'One-hot encoding of {col}')
      one_hot_cols = pd.get_dummies(sba[col])
      for ohc in one_hot_cols.columns:
        sba[col + '_' + ohc] = one_hot_cols[ohc]
    else:
      print(f'Hashing of {col}')
      sba[col + '_hash'] = sba[col].apply(lambda row: int(hashlib.sha1((col + "_" + \
                                    str(row)).encode('utf-8')).hexdigest(), 16) % hash_constant)

    cols_to_drop.append(col)
print(cols_to_drop)

In [158]:
sba.head(2)[['City','State','CityState','City_hash','State_hash','CityState_hash']]

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>TimeFrame</b><br>
Save a dataset for later use where we restrict the time frame to loans by excluding those disbursed after 2010 due to the fact the term of a loan is frequently 5 or more years.
    <br><br>

In [159]:
sba_bef_2011 = sba[sba['DisbursementDate'] <= '2010-12-31']
len(sba_bef_2011[sba_bef_2011['DisbursementDate'] > '2010-12-31'])

In [160]:
# Save 4
## save this dataset to working dir
sba_bef_2011.to_csv(savepath + "sba_bef_2011.csv",index=False)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Drop more columns that are no longer needed<b></div>

In [161]:
sba.drop(columns = ['State', 'City','CityState','ChgOffPrinGr',\
                    'DisbursementDate'], inplace = True)  
sba.head(2)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for Infinite Values<b></div>

In [162]:
def check_infinity_nan(df,dfname):
    print("checking for infinity")
  
    #ds = sba.isin([np.inf, -np.inf])
    #print(ds)
  
    # printing the count of infinity values
    print()
    print("printing the count of infinity values")
  
    count = np.isinf(df).values.sum()
    print(f"{dfname} contains " + str(count) + " infinite values")
    print()
    
    has_nan = df.isnull().values.any()
    print(f"Does {dfname} have Nan or Null values ?  {has_nan}")

In [163]:
check_infinity_nan(sba,'sba')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check Correlations</b></div>

In [164]:
fig, ax = plt.subplots(figsize=(20,20))

g = sns.heatmap(
    sba.corr(),
    annot=True,
    ax=ax,
    cmap='OrRd',
    cbar=False,
    linewidth=1
)

g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')

In [165]:
# Save 5
sba.to_csv(savepath + 'sba_save5.csv',index=False)

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>3. Build Model Using XGBoost</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>3.1 Prepare Data</h2>
    </div>

In [106]:
# Select subset of predictors
X = sba.copy()

# Select target
y = X.pop('MIS_Status')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Split Data into Train:Validate:Test<b></div>

In [168]:
from sklearn.model_selection import train_test_split

# Let's split the data in 80:10:10 for train:valid:test dataset
train_size=0.8 

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8) 

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem,test_size=0.5,random_state=101) 

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape) 

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Metrics Function

In [169]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix

def model_eval(y_valid,predictions):
    print('MAE:', metrics.mean_absolute_error(y_valid, predictions))
    print('MSE:', metrics.mean_squared_error(y_valid, predictions))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, predictions)))
    print()
    
    print("Classification Report:")
    print(classification_report(y_valid,predictions.round()))
    print()
    
    print("\033[1m" + "Accuracy for model:\033[0m %.2f" % (accuracy_score(y_valid, \
                                                            predictions.round()) * 100))

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>3.1 Model 1</h2>
    </div>

In [105]:
from xgboost import XGBRegressor

my_model1 = XGBRegressor()
my_model1.fit(X_train, y_train)

# wait a bit to complete

NameError: name 'X_train' is not defined

In [171]:
predictions1 = my_model1.predict(X_valid)

In [172]:
model_eval(y_valid, predictions1)

**High Accuracy for model, but precision, recall, and f1-score for classification 0 (CHGOFF) is not good, compared to that of classification 1 (P I F)**

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Improve Model 1</b></div>

In [173]:
# In the first step we will split the data in training and remaining dataset
X_train2, X_rem, y_train2, y_rem = train_test_split(X, y, train_size=0.8) 

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5

X_valid2, X_test, y_valid2, y_test = train_test_split(X_rem,y_rem,test_size=0.5,random_state=101) 

In [174]:
my_model1 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model1.fit(X_train2, y_train2, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid2, y_valid2)], 
             verbose=False)

# Wait, will take time

In [175]:
# Get predictions
predictions2 = my_model1.predict(X_valid2)

In [176]:
print("\033[4m" + "After Improving Model:" + "\033[0m")
model_eval(y_valid2, predictions2)
print()

print("\033[4m" + "Before Improving Model:" + "\033[0m")
model_eval(y_valid, predictions1)

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Not much difference between initial build and improved build.<br><br>

Precision, recall, and f1-score of classification 0 (CHGOFF) is much lower than that of 1 (P I F). This is because MIS_Status is heavily skewed towards 1 (P I F).  To solve this, we try Oversampling the data, in the next section.</b></div>

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>3.2 OverSample</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;"><h2>3.2.1 Model 2</h2>
    </div>

In [ ]:
!pip install imbalanced-learn

In [177]:
# Select subset of predictors
X2 = sba.copy()

# Select target
y2 = X2.pop('MIS_Status')

In [178]:
from imblearn.over_sampling import RandomOverSampler

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

In [179]:
from collections import Counter

# fit and apply the transform
X_over, y_over = oversample.fit_resample(X2, y2)

# summarize class distribution
print('Before : ',Counter(y2))
print('After  : ',Counter(y_over))

In [180]:
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X_over, y_over, train_size=0.8) 

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem,test_size=0.5,random_state=101) 

In [181]:
from xgboost import XGBRegressor

my_model2 = XGBRegressor()
my_model2.fit(X_train, y_train)

# wait a bit to complete

In [182]:
predictions3 = my_model2.predict(X_valid)

In [183]:
model_eval(y_valid, predictions3)

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>After oversampling of the minority class (CHGOFF), class 0 (CHGOFF) now has similar precision, recall, and f1-score as class 1 (P I F)</b>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Improve The Model</b></div>

In [184]:
my_model2 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model2.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Wait a bit, will take time

In [185]:
# Get predictions
predictions4 = my_model2.predict(X_valid)

In [186]:
model_eval(y_valid, predictions4)

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>We get a tiny improvement.  I didn't try other parameters for XGBRegressor yet.</b></div>

In [216]:
# Plot feature importance
from xgboost import plot_importance
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize,dpi=600)
    return plot_importance(booster=booster, ax=ax)

plot_features(my_model2, (10,14))

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Observation</b><br>
    I was hoping to see <b>Industry</b> at a much higher position here, but apparently the incomplete data on industry had an effect.<br><br>
Furthermore, <b>Recession</b> has to be at a very high position, but is at the bottom instead.  This could be due to <b>Recession</b> data being highly skewed towards 1 (Not Recession).<br><br>

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>3.2.2 Model 3</h2>
    <b>Build a Model Dataset Excluding Year 2011 and Above</b>

We restrict the time frame to loans by excluding those disbursed after 2010 due to the fact the term of a loan is frequently 5 or more years.
       </div>

In [107]:
sba_df = pd.read_csv(savepath + "sba_bef_2011.csv")
sba_df = sba_df.drop(columns = ['State', 'City','CityState',\
                                        'ChgOffPrinGr','DisbursementDate']) 
print(sba_df.columns)

# Select subset of predictors
X = sba_df.copy()

# Select target
y = X.pop('MIS_Status')

Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'LowDoc', 'DisbursementGross',
       'MIS_Status', 'GrAppv', 'SBA_Appv', 'Industry', 'Recession',
       'RealEstate', 'SBA_Portion', 'City_hash', 'State_hash',
       'CityState_hash'],
      dtype='object')


In [108]:
sba_df.head(2)

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementGross,MIS_Status,GrAppv,SBA_Appv,Industry,Recession,RealEstate,SBA_Portion,City_hash,State_hash,CityState_hash
0,84,4,2,0,0,1,0,1,60000.0,1,60000.0,48000.0,45,0,0,80.0,74511,608543,827599
1,60,2,2,0,0,1,0,1,40000.0,1,40000.0,32000.0,72,0,0,80.0,274452,608543,391234


In [109]:
from imblearn.over_sampling import RandomOverSampler

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

In [111]:
from collections import Counter

# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)

# summarize class distribution
print('Before : ',Counter(y))
print('After  : ',Counter(y_over))

Before :  Counter({1: 714215, 0: 154479})
After  :  Counter({1: 714215, 0: 714215})


In [113]:
from sklearn.model_selection import train_test_split

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X_over, y_over, train_size=0.8) 

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem,test_size=0.5,random_state=101) 

In [ ]:
my_model3 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model3.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Wait a bit, will take time

In [194]:
# Get predictions
predictions5 = my_model3.predict(X_valid)

In [195]:
model_eval(y_valid, predictions5)

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Will use this dataset as the last dataset, for now.</b></div>

In [217]:
# Plot feature importance

plot_features(my_model3, (10,14))

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>4. Test Model</h2>
    </div>
    

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>4.1 Test Model with Test Dataset</h2>
    </div>

In [197]:
# Get predictions
predictions6 = my_model3.predict(X_test)

In [198]:
model_eval(y_test, predictions6)

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>4.2 Test Model with User Input</h2>
    </div>

**So let's assume the following are the entries of a user, through a user interface, looking for a prediction from our model**

In [199]:
'''
17 Entries :
'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'LowDoc', 'DisbursementGross',
       GrAppv', 'SBA_Appv', 'Industry', 'Recession','RealEstate', 'SBA_Portion', 'City', 'State'
'''

user_input = [2,0,1,0,0,1,0,0,50000,50000,25000,71,0,0,50,'EVANSVILLE','IN']

city = user_input[15]
state = user_input[16]
city_state = city + "_" + state

city_hash = int(hashlib.sha1(('City' + "_" + \
                              str(city)).encode('utf-8')).hexdigest(), 16) % 900000
state_hash = int(hashlib.sha1(('State' + "_" + \
                              str(state)).encode('utf-8')).hexdigest(), 16) % 900000
city_state_hash = int(hashlib.sha1(('CityState' + "_" + \
                              str(city_state)).encode('utf-8')).hexdigest(), 16) % 900000

print(city_hash)
print(state_hash)
print(city_state_hash)

user_input[15] = city_hash
user_input[16] = state_hash
user_input.append(city_state_hash)

print(user_input)

In [200]:
features = np.array([user_input])   

# using inputs to predict the output
pred = my_model3.predict(features)
print("Prediction: {}".format(pred))

**Predictions:**<br>
- near 1 -> can approve
- lower than 0.5 -> do not approve

Of course, in real life, will need to check further using other data or models if available.

<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2>5. Mutual Information Scores</h2>
 A general-purpose metric, ideally used before selecting and building a model, but used here in the end, for comparison.
    </div>

In [201]:
def make_mi_scores(X, y):
    X = X.copy()
    #for colname in X.select_dtypes(["object", "category"]):
    #    X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    #discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    #mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = mutual_info_regression(X, y, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [222]:
from sklearn.feature_selection import mutual_info_regression

mi_scores = make_mi_scores(X_over, y_over)

# Wait for some time

In [1]:
plt.figure(dpi=1200, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [218]:
# Plot feature importance

plot_features(my_model3, (10,14))